In [1]:
import sys
import os
import pandas as pd
from tqdm import tqdm
sys.path.insert(1, os.path.join(sys.path[0], '..'))
from mta_metrics import bert_sim, \
    bert_contained_in, chrp_contained_in
from mta_metrics.alignment import awesome_align, usw_nm
from translate import gpt4, nllb, m2m
tqdm.pandas()

/Users/chany/miniforge3/envs/py3.8/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('inner.csv')
df.columns

Index(['idiom', 'meaning', 's_f_brkt', 's_l_brkt', 's_f', 's_l', 's_a',
       't_f_brkt_ko', 't_l_brkt_ko', 't_f_ko', 't_l_ko', 't_f_a_ko',
       't_l_a_ko', 't_f_brkt_zh', 't_l_brkt_zh', 't_f_zh', 't_l_zh',
       't_f_a_zh', 't_l_a_zh'],
      dtype='object')

# Korean

### Translate

In [3]:
df['p_f_ko_gpt4'] = df['s_f'].progress_apply(lambda x: gpt4(x, 'Korean'))
df['p_l_ko_gpt4'] = df['s_l'].progress_apply(lambda x: gpt4(x, 'Korean'))
df['p_a_ko_gpt4'] = df['s_a'].progress_apply(lambda x: gpt4(x, 'Korean'))

100%|██████████| 87/87 [03:51<00:00,  2.66s/it]


In [5]:
df['p_f_ko_nllb'] = nllb(df['s_f'].tolist(), lang='ko')
df['p_l_ko_nllb'] = nllb(df['s_l'].tolist(), lang='ko')
df['p_a_ko_nllb'] = nllb(df['s_a'].tolist(), lang='ko')

/Users/chany/miniforge3/envs/py3.8/lib/python3.8/site-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (200) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/Users/chany/miniforge3/envs/py3.8/lib/python3.8/site-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (200) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/Users/chany/miniforge3/envs/py3.8/lib/python3.8/site-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (200) to control the generation length. This behaviour is deprecated and will be removed from the config

In [7]:
df.to_csv("ko.csv", index=False)

## Sample Translation: Spanish

## USW, NM

In [13]:
alignments_a = awesome_align(df['s_a'], df['p_a'])
alignments_f = awesome_align(df['s_f'], df['p_f'])
alignments_l = awesome_align(df['s_l'], df['p_l'])
df['usw_a'], df['nm_a'], usw_a, nm_a = usw_nm(df['s_a'], alignments_a, df['p_a'])
df['usw_f'], df['nm_f'], usw_f, nm_f = usw_nm(df['s_f'], alignments_f, df['p_f'])
df['usw_l'], df['nm_l'], usw_l, nm_l = usw_nm(df['s_l'], alignments_l, df['p_l'])

Loading dataset...


Extracting alignments: 100%|██████████| 50/50 [00:00<00:00, 60.27it/s]


File ./source.txt.target.txt has been removed successfully
File ./source.txt has been removed successfully
File ./target.txt has been removed successfully
Loading dataset...


Extracting alignments: 100%|██████████| 50/50 [00:01<00:00, 44.01it/s]


File ./source.txt.target.txt has been removed successfully
File ./source.txt has been removed successfully
File ./target.txt has been removed successfully
Loading dataset...


Extracting alignments: 100%|██████████| 50/50 [00:01<00:00, 45.26it/s]


File ./source.txt.target.txt has been removed successfully
File ./source.txt has been removed successfully
File ./target.txt has been removed successfully


In [14]:
print(usw_a, usw_f, usw_l)
print(nm_a, nm_f, nm_l)

11.6 10.966810966810966 7.932011331444759
7.557041446208113 5.977530322159433 4.640564034114135


## Contained In

In [9]:
df['contained_in(p_a,p_f)_BERT'] = bert_score_contained_in(df['p_a'], df['p_f'], 'sp')
df['contained_in(p_a,p_l)_BERT'] = bert_score_contained_in(df['p_a'], df['p_l'], 'sp')
df['contained_in(p_a,p_f)_chrP'] = chrp_contained_in(df['p_a'], df['p_f'])
df['contained_in(p_a,p_l)_chrP'] = chrp_contained_in(df['p_a'], df['p_l'])
df['sensitivity_BERT'] = abs(df['contained_in(p_a,p_l)_BERT'] - df['contained_in(p_a,p_f)_BERT'])
df['sensitivity_chrP'] = abs(df['contained_in(p_a,p_l)_chrP'] -  df['contained_in(p_a,p_f)_chrP'])

calculating scores...
computing bert embedding.


100%|██████████| 1/1 [00:03<00:00,  3.47s/it]


computing greedy matching.


100%|██████████| 1/1 [00:00<00:00, 178.96it/s]


done in 3.48 seconds, 2.88 sentences/sec
calculating scores...
computing bert embedding.


100%|██████████| 1/1 [00:02<00:00,  2.70s/it]


computing greedy matching.


100%|██████████| 1/1 [00:00<00:00, 218.15it/s]

done in 2.71 seconds, 3.69 sentences/sec


In [1]:
df

NameError: name 'df' is not defined